### Atividade 

#### Parte Ivan

- Criar arquivo terraform para criação de 3 buckets (raw, trusted e refined) e lambda para enviar arquivos de Raw para Trusted 
- Criar código para subir dados para bucket 
- criar código para enviar de lambda raw para trusted 
- Criar a rotina para reprocessamento
- Entregar arquivo PDF com a descrição dos tratamento e particionamentos realizados 
- Entregar arquivo ZIP com os dados Tratados
- Entregar arquivo ZIP com o script Python utilizado




#### Parte Gustavo e Luis

1) Separar quais arquivos cada um vai tratar

2) Analisar os Arquivos e considerar as transformações necessárias para limpeza,
padronização, enriquecimento de dados e reprocessamento dos dados 
- Sugestão de padronização no sumário abaixo (podém mudar o que preferirem, só alinhem entre si pois os arquivos tem que ter o mesmo tratamento)
- Converter dados de latitude e longitude que estão em graus para decimais (se for o arquivo que tem essa coluna, o professor disponibilizou o codigo)
- Somente considerar voos domésticos
 
3) Os arquivos de Voos e de Reclamações precisam passar por tratamento para que
sejam armazenados no Bucket Trusted de forma que facilitem reprocessamentos
4) Particionar por mes, ou seja, criar um arquivo csv para cada mes, nesse formato que o professor pediu : 
- dados_voo_YYYYMM
- reclamação_YYYYMM


### Avisos:

- Os arquivos csv são muito grandes para mandar p github então podem colocar em uma pasta chamada "dados", eu adicionei um .env que não vai mandar para o git, eu colocaria os dados em uma subpasta chamada raw e mandaria para outra subpasta chamada trusted, mas podem deixar que eu arrumo no final


- Para acessar e rodar os arquivos ipynb usar a extensão do VSCode jupyter notebook da Microsoft
- Esta Prática servirá de base para a nossa próxima aula
- Para o tratamento dos dados podem usar como base o nosso arquivo tratamento.ipynb no github https://github.com/BeiraMar-G2/beira-mar-data-analytics tem muita coisa que pode ser reaproveitada
- A Atividade pode ser feita em Grupo, não esqueça de informar Nome e RA
- Entrega
- Sugestão para nome de arquivo VOO_DOM_YYYYMM
- Não esqueça de documentar também o reprocessamento dos arquivos dos meses
de Novembro e Dezembro de 2024

### Sumário de padrões para tratamento de dados:

- Data: DD/MM/YYYY

- Data hora: DD/MM/YYYY HH:MM:SS

- Binário: 0 ou 1

- Strings e colunas: Maiusculas e sem acento: 

- Espaços: Manter apenas um espaço no intervalo entre palavras

- Sexo: M e F

- Delimitador: Vírgula


In [1]:
import math
import pandas as pd 
import win32com.client as win32
import unicodedata

In [ ]:
def converter_xls_para_xlsx(arquivo_xls, arquivo_xlsx):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(arquivo_xls)
    wb.SaveAs(arquivo_xlsx, FileFormat=51)  # 51 = formato xlsx
    wb.Close()
    excel.Application.Quit()

# Exemplo de uso
converter_xls_para_xlsx(
    r"C:\Users\demor\Documents\analise-de-voos\dados\Dados Atividade\aerodromospublicos.xls",
    r"C:\Users\demor\Documents\analise-de-voos\dados\Dados Atividade\aerodromospublicos.xlsx"
)

com_error: (-2147352567, 'Exceção.', (0, 'Microsoft Excel', 'Não foi possível encontrar C:\\Users\\demor\\Documents\\analise-de-voos\\dados\\Dados Atividade\\aerodromospublicos.xls. É possível que ele tenha sido movido, renomeado ou excluído?', 'xlmain11.chm', 0, -2146827284), None)

In [63]:
def dms_para_dd(graus, minutos, segundos, direcao):
    """
    Converte coordenadas de Graus/Minutos/Segundos (DMS) para Graus Decimais (DD).
    Exemplo: dms_para_dd(8, 20, 55, 'S') -> -8.348611
    """
    decimal = graus + (minutos / 60) + (segundos / 3600)
    if direcao in ['S', 'W']:
        decimal = -decimal
    return decimal


def haversine(lat1, lon1, lat2, lon2):
    """
    Calcula a distância entre dois pontos na Terra (em km)
    a partir de suas coordenadas em graus decimais.
    """
    R = 6371.0  # raio médio da Terra em km
    
    # Converter para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    
    return R * c


In [64]:
def remover_acentos(texto):
    """Remove acentos de um texto e converte para maiúsculas"""
    if isinstance(texto, str):
        # Normaliza o texto (decompõe caracteres acentuados)
        texto = unicodedata.normalize('NFKD', texto)
        # Remove os diacríticos (acentos)
        texto = u"".join([c for c in texto if not unicodedata.combining(c)])
        # Converte para maiúsculas
        return texto.upper()
    return texto

# Carregar os dados
aeroportos = pd.read_excel(r"C:\Users\demor\Documents\analise-de-voos\dados\Dados Atividade\aerodromospublicos.xlsx", engine="openpyxl", skiprows=2)
empresas = pd.read_excel(r"C:\Users\demor\Documents\analise-de-voos\dados\Dados Atividade\Empresas Aereas.xlsx", engine="openpyxl")

# Padronizar nomes das colunas (maiúsculo, sem acentos, sem espaços)
aeroportos.columns = [remover_acentos(col.strip().replace(" ", "_")) for col in aeroportos.columns]
empresas.columns = [remover_acentos(col.strip().replace(" ", "_")) for col in empresas.columns]

# Aplicar a função em todas as colunas do tipo objeto (texto)
for col in aeroportos.select_dtypes(include=['object']).columns:
    aeroportos[col] = aeroportos[col].apply(remover_acentos)

for col in empresas.select_dtypes(include=['object']).columns:
    empresas[col] = empresas[col].apply(remover_acentos)

print("\nColunas do DataFrame aeroportos:")
print(aeroportos.columns.tolist())
print("\nColunas do DataFrame empresas:")
print(empresas.columns.tolist())

print("\nExemplo dos dados após remoção de acentos:")
print("\nAeroportos:")
print(aeroportos.head())
print("\nEmpresas:")
print(empresas.head())


Colunas do DataFrame aeroportos:
['CODIGO_OACI', 'CIAD', 'NOME', 'MUNICIPIO_ATENDIDO', 'UF', 'LATITUDE', 'LONGITUDE', 'ALTITUDE', 'OPERACAO', 'OACI', 'AIP_BRASIL', 'DESIGNACAO', 'COMPRIMENTO', 'LARGURA', 'RESISTENCIA', 'SUPERFICIE', 'DESIGNACAO.1', 'COMPRIMENTO.1', 'LARGURA.1', 'RESISTENCIA.1', 'SUPERFICIE.1', 'DESIGNACAO.2', 'COMPRIMENTO.2', 'LARGURA.2', 'RESISTENCIA.2', 'SUPERFICIE.2', 'RAMPA(S)_DE_APROXIMACAO', 'FORMATO_DA_AREA_DE_POUSO', 'DIMENSOES', 'RESISTENCIA.3', 'SUPERFICIE.3', 'P1', 'P2', 'P3', 'P4', 'PORTARIAS', 'UNNAMED:_36', 'UNNAMED:_37', 'UNNAMED:_38', 'PORTARIA_NO_16.739/2025_E_ALTERACOES', 'PORTARIA_NO_11.183/2023\nA_CLASSE_I_SE_DIVIDE_AINDA_EM_RBAC_121,_RBAC_135_REGULAR_E_OUTRAS_OPERACOES', 'PORTARIA_NO_2007/2014_E_ALTERACOES\n(*_=_REGIME_TARIFARIO_DIFERENCIADO)', 'AERONAVE_CRITICA', 'TIPO_DE_APROX._(CABECEIRAS)', 'FREQ._SEMANAL', 'R1', 'R2', 'REFERENCIA(S)', 'RESTRICAO(OES)', 'SITUACAO', 'R1.1', 'REFERENCIA', 'SITUADO_NA_AMAZONIA_LEGAL', 'OBSERVACOES']

Colunas do D

In [66]:
# Função auxiliar para extrair valores de coordenadas em formato DMS
def extrair_dms(coord_str):
    """Extrai graus, minutos, segundos e direção de uma string no formato '8° 20' 55'' S'"""
    import re
    
    # Extrair números e direção
    match = re.match(r"(\d+)° (\d+)' (\d+)'' ([NSWE])", coord_str)
    if match:
        graus = int(match.group(1))
        minutos = int(match.group(2))
        segundos = int(match.group(3))
        direcao = match.group(4)
        return dms_para_dd(graus, minutos, segundos, direcao)
    return None

# Converter coordenadas sobrescrevendo as colunas existentes
if "LATITUDE" in aeroportos.columns and "LONGITUDE" in aeroportos.columns:
    aeroportos["LATITUDE"] = aeroportos["LATITUDE"].apply(extrair_dms)
    aeroportos["LONGITUDE"] = aeroportos["LONGITUDE"].apply(extrair_dms)
    
print("\nPrimeiros registros com coordenadas convertidas:")
print(aeroportos[["NOME", "LATITUDE", "LONGITUDE"]].head())


Primeiros registros com coordenadas convertidas:
                          NOME   LATITUDE  LONGITUDE
0        CONCEICAO DO ARAGUAIA  -8.348611 -49.303056
1                BAURU/AREALVA -22.157778 -49.068333
2         BARTOLOMEU DE GUSMAO -21.811944 -48.133056
3                  SANTA MARIA -10.985278 -37.073333
4  PILOTO OSVALDO MARQUES DIAS  -9.866389 -56.105000


In [67]:
if "tipo_voo" in empresas.columns:
    empresas_domesticas = empresas[empresas["tipo_voo"] == "DOMESTICO"]
else:
    empresas_domesticas = empresas.copy()

In [68]:
if {"latitude", "longitude"}.issubset(aeroportos.columns):
    # Usar as coordenadas convertidas
    lat1 = aeroportos.loc[0, "latitude"]
    lon1 = aeroportos.loc[0, "longitude"]
    lat2 = aeroportos.loc[1, "latitude"]
    lon2 = aeroportos.loc[1, "longitude"]
    
    distancia = haversine(lat1, lon1, lat2, lon2)
    print(f"Distância entre {aeroportos.loc[0,'nome']} e {aeroportos.loc[1,'nome']}: {distancia:.2f} km")

In [70]:
# Debugging: verificar os valores e tipos de dados
print("Tipos de dados das colunas:")
print(aeroportos[["LATITUDE", "LONGITUDE"]].dtypes)
print("\nPrimeiros registros:")
print(aeroportos[["NOME", "LATITUDE", "LONGITUDE"]].head())

Tipos de dados das colunas:
LATITUDE     float64
LONGITUDE    float64
dtype: object

Primeiros registros:
                          NOME   LATITUDE  LONGITUDE
0        CONCEICAO DO ARAGUAIA  -8.348611 -49.303056
1                BAURU/AREALVA -22.157778 -49.068333
2         BARTOLOMEU DE GUSMAO -21.811944 -48.133056
3                  SANTA MARIA -10.985278 -37.073333
4  PILOTO OSVALDO MARQUES DIAS  -9.866389 -56.105000


In [71]:
aeroportos.to_excel("aeroportos_tratados.xlsx", index=False)
empresas_domesticas.to_excel("empresas_domesticas_tratadas.xlsx", index=False)

print("✅ Tratamento concluído! Arquivos exportados:")
print("- aeroportos_tratados.xlsx")
print("- empresas_domesticas_tratadas.xlsx")

✅ Tratamento concluído! Arquivos exportados:
- aeroportos_tratados.xlsx
- empresas_domesticas_tratadas.xlsx


## Tratamento - dadosconsumidor2024.csv e VRA_2024.csv



In [9]:
# Importações e função auxiliar
import pandas as pd
import unicodedata

def limpar_texto(texto):
    """Remove acentos, converte para maiúscula e limpa espaços"""
    if not isinstance(texto, str) or texto == 'nan':
        return texto
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join(c for c in texto if not unicodedata.combining(c))
    return texto.upper().strip()

print("✅ Bibliotecas carregadas e função auxiliar definida!")

✅ Bibliotecas carregadas e função auxiliar definida!


In [6]:
# TRATAMENTO - RECLAMAÇÕES
print(" Processando reclamações...")

# Carregar e tratar em uma etapa
reclamacoes = pd.read_csv(
    r"c:\Users\Windows Pro\OneDrive - SPTech School\4° Semestre\dadosconsumidor2024.csv", 
    sep=';', encoding='utf-8', skiprows=1
)

# Padronizar colunas e dados
reclamacoes.columns = [limpar_texto(col.replace(' ', '_')) for col in reclamacoes.columns]

# Aplicar limpeza apenas em colunas de texto (exceto datas)
for col in reclamacoes.select_dtypes(include=['object']).columns:
    if not any(x in col.lower() for x in ['data', 'hora']):
        reclamacoes[col] = reclamacoes[col].apply(limpar_texto)

# Converter datas
date_cols = [col for col in reclamacoes.columns if 'DATA' in col]
for col in date_cols:
    if 'HORA' in col:
        reclamacoes[col] = pd.to_datetime(reclamacoes[col], errors='coerce').dt.strftime('%d/%m/%Y %H:%M:%S')
    else:
        reclamacoes[col] = pd.to_datetime(reclamacoes[col], errors='coerce').dt.strftime('%d/%m/%Y')

# Filtrar apenas dados aéreos
aereo = reclamacoes[(reclamacoes['AREA'] == 'TRANSPORTES') & (reclamacoes['ASSUNTO'] == 'AEREO')].copy()

print(f" {len(aereo)} reclamações aéreas processadas")

 Processando reclamações...
 82136 reclamações aéreas processadas
 82136 reclamações aéreas processadas


In [ ]:
# TRATAMENTO COMPLETO - VOOS
print(" Processando voos...")

# Carregar voos 
voos = pd.read_csv(
    r"c:\Users\Windows Pro\OneDrive - SPTech School\4° Semestre\VRA_2024.csv", 
    sep=';', encoding='utf-8', nrows=100000  # Limitar para demonstração
)

# Padronizar colunas
voos.columns = [limpar_texto(col.replace(' ', '_')) for col in voos.columns]

# Aplicar limpeza em colunas de texto (exceto datas)
for col in voos.select_dtypes(include=['object']).columns:
    if not any(x in col.lower() for x in ['partida', 'chegada', 'prevista', 'real']):
        voos[col] = voos[col].apply(limpar_texto)

# Converter datas/horas
datetime_cols = [col for col in voos.columns if any(x in col.upper() for x in ['PARTIDA', 'CHEGADA'])]
for col in datetime_cols:
    voos[col] = pd.to_datetime(voos[col], errors='coerce').dt.strftime('%d/%m/%Y %H:%M:%S')

# Filtrar voos domésticos (códigos ICAO brasileiros começam com 'SB')
domesticos = voos[
    (voos['SIGLA_ICAO_AEROPORTO_ORIGEM'].str.startswith('SB', na=False)) &
    (voos['SIGLA_ICAO_AEROPORTO_DESTINO'].str.startswith('SB', na=False))
].copy()

print(f" {len(domesticos)} voos domésticos processados ({len(domesticos)/len(voos)*100:.1f}%)")

 Processando voos...


C:\Users\Windows Pro\AppData\Local\Temp\ipykernel_7924\3055306298.py:5: DtypeWarning: Columns (3,6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  voos = pd.read_csv(
C:\Users\Windows Pro\AppData\Local\Temp\ipykernel_7924\3055306298.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  voos[col] = pd.to_datetime(voos[col], errors='coerce').dt.strftime('%d/%m/%Y %H:%M:%S')
C:\Users\Windows Pro\AppData\Local\Temp\ipykernel_7924\3055306298.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  voos[col] = pd.to_datetime(voos[col], errors='coerce').dt.strftime('%d/%m/%Y %H:%M:%S')
C:\Users\Windows Pro\AppData\Local\Temp\ipykernel_7924\3055306298.py:21: UserWarning: Could not infer forma

 78640 voos domésticos processados (78.6%)


In [ ]:
# PARTICIONAMENTO MENSAL
print(" Iniciando particionamento mensal...")

def particionar_por_mes(df, nome_base, coluna_data, formato_data='%d/%m/%Y'):
    """Função para particionar dados por mês"""
    # Converter data e extrair ano-mês
    df_temp = df.copy()
    df_temp['DATA_TEMP'] = pd.to_datetime(df_temp[coluna_data], format=formato_data, errors='coerce')
    df_temp = df_temp.dropna(subset=['DATA_TEMP'])
    df_temp['ANO_MES'] = df_temp['DATA_TEMP'].dt.strftime('%Y%m')
    
    # Gerar arquivos por mês
    arquivos = []
    for mes in sorted(df_temp['ANO_MES'].unique()):
        dados_mes = df_temp[df_temp['ANO_MES'] == mes].drop(['DATA_TEMP', 'ANO_MES'], axis=1)
        nome_arquivo = f'{nome_base}_{mes}.csv'
        dados_mes.to_csv(nome_arquivo, index=False, sep=',', encoding='utf-8')
        arquivos.append(nome_arquivo)
        print(f" {nome_arquivo}: {len(dados_mes)} registros")
    
    return arquivos

# Particionar reclamações
coluna_data_rec = next((col for col in aereo.columns if 'DATA' in col), None)
rec_files = particionar_por_mes(aereo, 'reclamacao', coluna_data_rec)

# Particionar voos 
coluna_data_voo = 'PARTIDA_PREVISTA' if 'PARTIDA_PREVISTA' in domesticos.columns else 'PARTIDA_REAL'
voo_files = particionar_por_mes(domesticos, 'dados_voo', coluna_data_voo, '%d/%m/%Y %H:%M:%S')

print(f"\n RESUMO: {len(rec_files)} reclamações + {len(voo_files)} voos = {len(rec_files) + len(voo_files)} arquivos")